In [1]:
!pip install flask pyngrok fastapi

In [2]:
import getpass
import os
import threading

from flask import Flask
from pyngrok import ngrok, conf

print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken")
conf.get_default().auth_token = getpass.getpass()

app = Flask(__name__)
app.debug = True
UPLOAD_FOLDER = '/'
ALLOWED_EXTENSIONS = {'json'}

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(5000).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}/\"".format(public_url, 5000))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url


Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken
··········
 * ngrok tunnel "https://59a7-34-145-191-120.ngrok-free.app" -> "http://127.0.0.1:5000/"


In [3]:
import pandas as pd
from flask import request, jsonify
import random

In [4]:
d = {
    'name':'Linus',
    'surname':'Torvalds',
    'idade':54
}

In [5]:
@app.route('/')
def home():
  return """
  <h1> Desafio DIO retornar json inputado </h1>
  <a href="/input"> Clique para acessar o input </a>
  <br>
  <a href="/output"> Clique para acessar o output </a>
  """

In [6]:
import os
from flask import Flask, flash, request, redirect, url_for
from werkzeug.utils import secure_filename

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/input', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        # check if the post request has the file part
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
        # If the user does not select a file, the browser submits an
        # empty file without a filename.
        if file.filename == '':
            flash('No selected file')
            return redirect(request.url)
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            return redirect(url_for('download_file', name=filename))
    return '''
    <!doctype html>
    <title>Upload new File</title>
    <h1>Upload new File</h1>
    <form method=post enctype=multipart/form-data>
      <input type=file name=file>
      <input type=submit value=Upload>
    </form>
    '''

In [7]:
from flask import send_from_directory

@app.route('/output/<name>')
def download_file(name):
    return send_from_directory(app.config["UPLOAD_FOLDER"], name)

In [8]:
# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()